<div align="center">
    <img src="https://socialify.git.ci/julep-ai/julep/image?description=1&descriptionEditable=Build%20AI%20agents%20and%20workflows%20with%20a%20simple%20API&font=Source%20Code%20Pro&logo=https%3A%2F%2Fraw.githubusercontent.com%2Fjulep-ai%2Fjulep%2Fdev%2F.github%2Fjulep-logo.svg&owner=1&pattern=Solid&stargazers=1&theme=Auto" alt="julep" width="640" height="320" />
</div>

## Task: Research Assistant with Web Search Integration

### Overview

This task is designed to conduct comprehensive research on multiple user-provided topics. It leverages web search capabilities to gather up-to-date information and generates relevant keywords for each topic, streamlining the research process through efficient tool integration and parallel processing.

### Task Flow

1. **User Input**
   - User provides a list of research topics
   - Each topic is processed individually

2. **Web Search and Information Gathering**
   - Utilizes BraveWeb to conduct searches for each topic
   - Retrieves latest news and relevant information

3. **Intelligent Analysis**
   - System analyzes search results (HTML snippets)
   - Generates Wikipedia keywords related to each topic
   - AI assistant identifies most relevant keywords based on search content

4. **Parallel Processing**
   - Implements concurrent topic processing
   - Enhances efficiency and reduces overall research time

### Key Features

- **Multi-topic Research**: Handles multiple research topics simultaneously
- **Up-to-date Information**: Utilizes web search to access current data
- **Keyword Generation**: Provides relevant Wikipedia keywords for each topic
- **AI-assisted Analysis**: Employs AI to identify and prioritize key information
- **Efficient Processing**: Leverages parallel processing for faster results

### Output

- Comprehensive list of relevant Wikipedia keywords for each input topic
- Curated, up-to-date information gathered from web searches

This workflow provides a robust, efficient approach to in-depth topic exploration, combining the power of web search, AI analysis, and parallel processing.

```plaintext

+----------------+     +--------------------------+     +-----------------+     +------------------------+     +-------------------------+
|   User Input   |     | Web Search & Information |     |  Intelligent    |     |   Parallel Processing  |     |      Output Stage       |
|(List of Topics)| --> | Gathering (BraveWeb)     | --> |    Analysis     | --> | (Concurrent Execution) | --> |  Summarized Research    |
|                |     |                          |     |                 |     |                        |     |     (Final Report)      |
+----------------+     +--------------------------+     +-----------------+     +------------------------+     +-------------------------+
       |                              |                          |                           |                              |
       |                              |                          |                           |                              |
       v                              v                          v                           v                              v
  Topic 1, Topic 2, ...        Retrieve HTML snippets      Extract Wikipedia            Process multiple             Compile summary and 
  Each topic processed         from search results        keywords from HTML           topics concurrently          present most relevant 
  individually.                for each topic.              snippets, analyze            for faster results.          findings per topic.
                                                            relevance.
```


### Implementation

To recreate the notebook and see the code implementation for this task, you can access the Google Colab notebook using the link below:

<a target="_blank" href="https://colab.research.google.com/github/julep-ai/julep/blob/f/dev/cookbooks/03-SmartResearcher_With_WebSearch.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### Additional Information

For more details about the task or if you have any questions, please don't hesitate to contact the author:

**Author:** Julep AI  
**Contact:** [hey@julep.ai](mailto:hey@julep.ai) or  <a href="https://discord.com/invite/JTSBGRZrzj" rel="dofollow">Discord</a>

Installing the Julep Client

In [1]:
!pip install julep -U --quiet

#### NOTE:

- UUIDs are generated for both the agent and task to uniquely identify them within the system.
- Once created, these UUIDs should remain unchanged for simplicity.
- Altering a UUID will result in the system treating it as a new agent or task.
- If a UUID is changed, the original agent or task will continue to exist in the system alongside the new one.

In [1]:
# Global UUID is generated for agent and task
import uuid

AGENT_UUID = uuid.uuid4()
TASK_UUID = uuid.uuid4()

### Creating Julep Client with the API Key

In [4]:
from julep import Client

api_key = "" # Your API key here

# Create a client
client = Client(api_key=api_key, environment="dev")

## Creating an "agent"


Agent is the object to which LLM settings, like model, temperature along with tools are scoped to.

To learn more about the agent, please refer to the [documentation](https://github.com/julep-ai/julep/blob/dev/docs/julep-concepts.md#agent).

In [5]:
# Defining the agent
name = "Jarvis"
about = "The original AI conscious the Iron Man."
default_settings = {
    "temperature": 0.7,
    "top_p": 1,
    "min_p": 0.01,
    "presence_penalty": 0,
    "frequency_penalty": 0,
    "length_penalty": 1.0,
    "max_tokens": 150,
}


# Create the agent
agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name=name,
    about=about,
    model="gpt-4o",
)

## Defining a Task

Tasks in Julep are Github Actions style workflows that define long-running, multi-step actions. 
You can use them to conduct complex actions by defining them step-by-step. They have access to all Julep integrations.

To learn more about tasks, visit [Julep documentation](https://github.com/julep-ai/julep/blob/dev/docs/julep-concepts.md#task).

In [8]:
import yaml

Here is a Task which uses a agent to generate a sarcastic response to a given text using a DuckDuckGo and Wikipedia tool.

More on how to define a task can be found [here](https://github.com/julep-ai/julep/blob/dev/docs/julep-concepts.md).

In [9]:
task_def= yaml.safe_load("""
name: Research Assistant to find Wikipedia Keywords

input_schema:
  type: object
  properties:
    topics:
      type: array
      items:
        type: string
      description: The topics to search for.

tools:
- name: brave_search
  type: integration
  integration:
    provider: brave
    setup:
      api_key: "YOUR_API_KEY"

main:
- over: _.topics
  map:
    tool: brave_search
    arguments:
      query: "'the latest news about ' + _"

- over: _
  parallelism: 2
  map:
    prompt:
    - role: system
      content: >-
        You are a research assistant.
        I need you to do in-depth research on topics trending in the news currently.
        Based on the following latest html news snippet, come up with a list of wikipedia keywords to search:
        "{{_}}"
        Your response should be a list of keywords, separated by commas. Do not add any other text.
        Example: `KEYWORDS: keyword1, keyword2, keyword3`

    unwrap: true
""")

Creating/Updating a task to generate a sarcastic response to a given text using a Intergation.

In [10]:
# creating the task object
task = client.tasks.create_or_update(
    task_id=TASK_UUID,
    agent_id=AGENT_UUID,
    **task_def
)

## Creating an Execution

An execution is a single run of a task. It is a way to run a task with a specific set of inputs.

Creates a execution worflow for the Task defined in the yaml file.

In [12]:
execution= client.executions.create(
    task_id=task.id,
    input={
        "topics": ["Burger King Cup on the Ground Behind a Wendy’s","Forbidden Chemical X","Finger Bracelets","Amusing Notions"]
    }
)

In [ ]:
execution.id

In [22]:
# getting the execution details
execution = client.executions.get(execution.id)
#printing the outpu
execution.output

Retrieves and lists all the steps of a defined task that have been executed up to that point in time. Unlike streaming, this function does not continuously monitor the execution; it only provides a snapshot of the steps completed so far without displaying real-time updates as the task progresses.

In [ ]:
client.executions.transitions.list(execution_id=execution.id).items

Continuously monitor and stream the steps of a defined task. It retrieves and displays the transitions or execution steps of the task identified by execution.id in real-time, showing each step sequentially until the task is either completed or an error causes it to terminate.

In [ ]:
client.executions.transitions.stream(execution_id=execution.id)